In [ ]:
# Import the required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Load the dataframe with URLs from an Excel file
df = pd.read_excel('URL.xlsx')

# Print the shape of the dataframe to check the number of rows and columns
df.shape

# Create an empty list to store bad URLs
bad_url = []

# Define a function to extract the image URL from a URL
def extract_image_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    div = soup.find('div', {'class': 'ep-entity-header__main-image'})
    style = div['style']
    start_index = style.index('//')
    # Check the image format in the style attribute of the div tag and get the end index
    if 'jpg' in style:
        end_index = style.index('jpg') + 3
    elif 'png' in style:
        end_index = style.index('png') + 3
    elif 'jpeg' in style:
        end_index = style.index('jpeg') + 4
    # Return the complete image URL
    return 'https:' + style[start_index:end_index]

# Define a function to extract the flag image URL from a URL
def extract_flag_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    img_tag = soup.find('h1', {'class': 'ep-entity-header__name'}).find('img')
    img_url = img_tag['src']
    # Return the complete flag image URL
    return 'https:' + img_url

# Loop through all URLs in the 'URL' column of the dataframe
for i, row in df.iterrows():
    try:
        url = row['URL']
        # Extract the paragraph image URL and flag image URL using the defined functions
        para = extract_image_url(url)
        flag = extract_flag_url(url) 
        # Update the corresponding rows in the dataframe with the extracted URLs
        df.at[i, 'para'] = para
        df.at[i, 'flag'] = flag
        # Wait for 2 seconds before proceeding to the next URL
        time.sleep(2)
    except:
        # If an exception occurs, add the bad URL to the list
        row_data={'Bad_URL':row['URL']}
        bad_url.append(row_data)
        continue

# Create a new dataframe with the bad URLs
dx=pd.DataFrame(bad_url)

# Create a new dataframe with the rows that have a paragraph image URL
df1=df[df['para'].notnull()]

# Print the shape of the new dataframe to check the number of rows and columns
df1.shape 

# Save the updated dataframe to a new Excel file
df1.to_excel('URL2.xlsx', index=False)
